The following few cells setup the neural networks, provide a few helper functions, and connect to the jetbot's motors and camera.

In [ ]:
# Imports, do not modify 
import torch
import torchvision
import torch.nn.functional as F

import time
import cv2
import numpy as np
from jetbot import Robot, Camera, ObjectDetector

Setup the neural networks for detecting objects in the coco dataset, and determining if there will be a collision:

- Detect objects in the coco dataset: [ssd_mobilenet_v2_coco.engine](https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view). A list of the object ids can be found [here](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt).

- Pretrained collision avoidance network: [best_model.pth](https://drive.google.com/file/d/1UsRax8bR3R-e-0-80KfH2zAt-IyRPtnW/view).

In [ ]:
# Setting up models

# Setup collision avoidance model from best_model.pth
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

# For Part III, you do not need do not modify this line
# PART IV (i) and (ii), replace network line with <your_model>.pth, which you trained in google collab. 
model.load_state_dict(torch.load('best_model.pth'))

# Moving the model onto the gpu 
device = torch.device('cuda')
model = model.to(device)

# Setup coco model from ssd_mobilenet_v2_coco.engine
coco_model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
# Connecting to camera and robot, do not modify

# Need 300x300 image for coco dataset's NN, downsampled to 224 when using collision avoidance network
camera = Camera.instance(width=300, height=300)
robot = Robot()

In [ ]:
# Helper functions
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

def get_network_outputs(image):
    '''
    Runs the image through both networks, returns network outputs and a rescaled version of the image
    Example use if previous cells have been executed:
    
    prob_blocked, detections = get_network_outputs(camera.value)
    ''' 
    coco_x = image
    detections = coco_model(coco_x)

    compressed_image = cv2.resize(image, (224, 224))
    y = model(preprocess(compressed_image))
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])

    # Uncomment the following line when you are working on Part IV:
    # log(prob_blocked, compressed_image)
    
    return (prob_blocked, detections)    

In [ ]:
'''
This cell provides functions which you will need in Part IV. 
We ask that you to include the first person view (FPV) of your jetbot as well as a log of the network's output

Make sure to uncomment the log(prob_blocked, x) line in the get_network_outputs() function.
Example use if previous cells have been executed:

FPV_filename = 'PartIV_LogFPV'
prob_blocked_filename = 'PartIV_LogProbBlocked'
start_logger(FPV_filename)

for i in range(50):
    prob_blocked, detections = get_network_outputs(camera.value)  
    
    time.sleep(0.05)

stop_logger(prob_blocked_filename)
'''
video_logger = None 
prob_blocked_logger = None 

def start_logger(FPV_filename):
    global video_logger, prob_blocked_logger
    video_logger = cv2.VideoWriter(FPV_filename + '.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (224, 224))
    prob_blocked_logger = []

def log(prob_blocked, image):
    global video_logger, prob_blocked_logger
    video_logger.write(image)
    prob_blocked_logger.append(prob_blocked)
    
def stop_logger(prob_blocked_filename):
    global video_logger, prob_blocked_logger
    np.save(prob_blocked_filename, prob_blocked_logger)
    video_logger.release()
    cv2.destroyAllWindows()

## Implement your method below